In [1]:
print("Importing all Libraries")

import os
import gc
import time
import json
import numpy as np
import pandas as pd
import glob
import time
import matplotlib.pyplot as plt
from datetime import timedelta, date
import seaborn as sns
import torch
from fastai.tabular.all import *
import pickle
import datetime
import matplotlib.pyplot as plt
%matplotlib inline


Importing all Libraries


In [2]:
!pip install tensorflow

In [3]:
import io
import os
import re
import shutil
import string
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import Sequential, layers
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

from keras.preprocessing.text import one_hot
#from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Embedding
from keras.layers import Embedding

from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
from tensorflow.keras.layers import Embedding
from tensorflow.keras import layers


In [4]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)
pd.options.display.float_format='{:,.4f}'.format
pd.options.display.max_rows=200
pd.set_option('display.max_rows',800)
pd.set_option('display.max_columns',500)
pd.set_option('display.width',1000)

In [5]:
df1 = pd.read_parquet('/content/drive/MyDrive/Dresses_2022_23.parquet', engine='pyarrow')

In [6]:
# User Inputs

user_input = pd.read_csv('/content/drive/MyDrive/User_Input.csv')


# Input size mapping
size_mapping = pd.read_csv('/content/drive/MyDrive/Input_size_mapping.csv')

In [7]:
try:
  size_mapping.rename(columns={'International':'size_name'},inplace=True)
  size_mapping.columns = [x.lower() for x in size_mapping.columns]
except:
  print("No such column exists")

In [8]:
user_input.columns = [x.replace(' ','_') for x in user_input.columns]
user_input.columns = [x.lower() for x in user_input.columns]
user_input.head()

,user_id,sku_config,column_a,bust,waist,hips
0,2349207,BE000AA0E4B820GS,Hourglass,43.0000,37.0000,45.0000
1,119851,F3943AA1F007C2GS,Inverted triangle,NaN,NaN,NaN
2,8007973,63A0EAA74D97A6GS,Pear,NaN,NaN,NaN
3,3611307,B8103AA1B54DA8GS,Rectangle,NaN,NaN,NaN
4,1177389,E3C17AA35581FBGS,Round,NaN,NaN,NaN


In [9]:
user_input['user_id'] = user_input['user_id'].fillna(0).astype('int').astype('str')

In [10]:
user_input = pd.merge(user_input, size_mapping, left_on='column_a', right_on='image_name')
user_input = user_input[['user_id', 'sku_config', 'hips', 'waist', 'bust', 'gender', 'size_system_name', 'size_name']]
user_input.columns = ['user_id', 'sku_config', 'u_hips', 'u_waist', 'u_bust', 'u_gender', 'size_system_name', 'size_name']
user_input.columns = [x.lower() for x in user_input.columns]

user_input.head()

,user_id,sku_config,u_hips,u_waist,u_bust,u_gender,size_system_name,size_name
0,2349207,BE000AA0E4B820GS,45.0000,37.0000,43.0000,Women,International,S
1,2349207,BE000AA0E4B820GS,45.0000,37.0000,43.0000,Women,International,S/M
2,2349207,BE000AA0E4B820GS,45.0000,37.0000,43.0000,Women,International,M/L
3,2349207,BE000AA0E4B820GS,45.0000,37.0000,43.0000,Women,International,L
4,2349207,BE000AA0E4B820GS,45.0000,37.0000,43.0000,Women,International,XL


In [11]:
product = pd.read_parquet('/content/drive/MyDrive/Product_data.parquet', engine='pyarrow')
product.head()

,sku,sku_simple,size_name,size_system_name,measurements,bust,waist,hip,product_gender,brand_name,department_code
0,IN509AA81SXUSG,IN509AA81SXUSG-445250,XS,International,,38,31,41,Female,inner circle,RBO
1,IN509AA81SXUSG,IN509AA81SXUSG-445252,M,International,,43,37,45,Female,inner circle,RBO
2,FU985AA88KRZSG,FU985AA88KRZSG-538233,S,International,"Bust circumference x /> Waist circumference x Length /> - S (31.2\""\"" x 26.8\""\"" x 33.3\""\"") /> - M (33.8\""\"" x 28.4\""\"" x 33.9\""\"") /> - L (34.2\""\"" x 30\""\"" x 34.4\""\"") /> - XL (37\""\"" x 31.4\""\"" x 34.4\""\"") /> - XXL (37\""\"" x 33\""\"" x 35\""\"") /> - XXXL (39.4\""\"" x 34.6\""\"" x 35.4\""\"")",39.5,34,42,Female,future state,RBC
3,FU985AA88KRZSG,FU985AA88KRZSG-538234,M,International,"Bust circumference x /> Waist circumference x Length /> - S (31.2\""\"" x 26.8\""\"" x 33.3\""\"") /> - M (33.8\""\"" x 28.4\""\"" x 33.9\""\"") /> - L (34.2\""\"" x 30\""\"" x 34.4\""\"") /> - XL (37\""\"" x 31.4\""\"" x 34.4\""\"") /> - XXL (37\""\"" x 33\""\"" x 35\""\"") /> - XXXL (39.4\""\"" x 34.6\""\"" x 35.4\""\"")",43,36,44,Female,future state,RBC
4,FU985AA84KSDSG,FU985AA84KSDSG-538258,L,International,"Bust circumference x /> Waist circumference x Length /> - S (31.8\""\"" x 28.4\""\"" x 33.5\""\"") /> - M (33\""\"" x 28.8\""\"" x 34.3\""\"") /> - L (34.6\""\"" x 30\""\"" x 34.4\""\"") /> - XL (37.4\""\"" x 32.2\""\"" x 35\""\"") /> - XXL (37.8\""\"" x 34.6\""\"" x 35.6\""\"")",44,38,48,Female,future state,RBC


In [12]:
#user_input.rename(columns={'user_id':'id_customer'},inplace=True)
user_input = pd.merge(user_input, product, left_on=['sku_config', 'size_system_name', 'size_name'], right_on=['sku', 'size_system_name', 'size_name'])

In [13]:
# Scenario 1: User's product search is narrowed down basis least gap between body measurements and product measurements
user_input1 = user_input[user_input['u_bust'].notnull()].copy()
user_input1['gap'] = abs(user_input1['bust'].astype('int') - user_input1['u_bust'].astype('int')) + abs(user_input1['hip'].astype('int') - user_input1['u_hips'].astype('int')) + abs(user_input1['waist'].astype('int') - user_input1['u_waist'].astype('int'))
user_input2 = user_input1.groupby(['user_id', 'sku_config'])['gap'].min().reset_index()
user_input1 = pd.merge(user_input1, user_input2, on=['user_id', 'sku_config', 'gap'])
user_input1 = user_input1[['user_id', 'sku_config', 'size_name']].drop_duplicates()
user_input1.head()




,user_id,sku_config,size_name
0,2349207,BE000AA0E4B820GS,42


In [14]:
len(df1)

2831643

## **Scenario 2: User's previous purchase history is** **used**

In [15]:
df1 = df1[df1['product_gender']=='Female']


In [16]:
mask1 = (df1['bust'].notnull()) & (df1['waist'].notnull()) & (df1['hip'].notnull()) & (df1['bust']!= '') & (df1['waist']!= '') & (df1['hip']!='')

df1 = df1[mask1].copy()

In [17]:
len(df1)

328891

In [20]:
df1['id_customer'] = df1['id_customer'].fillna(0).astype('int')
df1['id_customer'].head()

12    1326209
29    4139843
50    1326209
51    1326209
60    4443492
Name: id_customer, dtype: int64

In [22]:
df1['record_keep'] = np.where(df1['outcome']=='net', 1,
                              np.where((df1['outcome']=='returned') & (df1['return_reason'].isin(['Size Too Small', 'Size Too Large'])),1,0))

In [23]:
print(len(df1))
df1 = df1[df1['record_keep']==1].copy()
print(len(df1))

328891
250620


In [24]:
df1['target_size_actual'] = np.where(df1['outcome']=='net', 'Fit',
                                     np.where((df1['outcome']=='returned') & (df1['return_reason']=='Size Too Small'), 'Small',
                                              np.where((df1['outcome']=='returned') & (df1['return_reason']=='Size Too Large'), 'Large', '')))

In [26]:
df1['target_size_actual'] = df1['target_size_actual'].str.lower()

In [27]:
df1['target_size_actual'].value_counts()

fit      191482
small     30628
large     28510
Name: target_size_actual, dtype: int64

In [28]:
df1['target_size'] = np.where(df1['target_size_actual']=='fit', 1,
                                     np.where(df1['target_size_actual']=='small',0,2))

In [29]:
cols = ['id_customer', 'customer_age', 'gender', 'venture_code',  'sku_config', 'sku_simple',
        'sub_category_type',  'size_name', 'size_system_name',
        'brand_name', 'product_gender', 'department_code', #'product_name', 'material', 'size_brand',
        'bust', 'waist', 'hip', 'target_size_actual', 'target_size']

df2 = df1[cols].drop_duplicates()

In [30]:
#df2.head().to_csv('Purchase_data.csv', index=None)

In [31]:
len(df2[df2['customer_age'].notnull()])/len(df2), len(df2[df2['gender'].notnull()])/len(df2)

(0.6172989236277128, 1.0)

In [32]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
df2['size_le'] = le.fit_transform(df2['size_name'])

In [33]:
df2['sku_le'] = le.fit_transform(df2['sku_config'])

In [34]:
df2 = df2.drop('customer_age',axis=1)


In [35]:
df2 = df2.dropna()
#len(df2), len(df3)

In [36]:
df2.head(2)

,id_customer,gender,venture_code,sku_config,sku_simple,sub_category_type,size_name,size_system_name,brand_name,product_gender,department_code,bust,waist,hip,target_size_actual,target_size,size_le,sku_le
12,1326209,female,sg,7CCB0AA409ED53GS,7CCB0AA409ED53GS-069NOG,Dresses,XL,International,Trendyol,Female,RBO,43,37,58.5,small,0,53,2654
29,4139843,female,my,9CA4FAAEC1DE3EGS,9CA4FAAEC1DE3EGS-046LBL,Dresses,XS/P,Brand Size,H&M,Female,MP,31,35,43.5,large,2,58,3361


In [37]:
cont,cat = cont_cat_split(df2, max_card=10, dep_var='target_size')
procs = [Categorify]


In [38]:
df2.dtypes

id_customer            int64
gender                object
venture_code          object
sku_config            object
sku_simple            object
sub_category_type     object
size_name             object
size_system_name      object
brand_name            object
product_gender        object
department_code       object
bust                  object
waist                 object
hip                   object
target_size_actual    object
target_size            int64
size_le                int64
sku_le                 int64
dtype: object

In [39]:
cont.remove('sku_le')
cat.remove('sku_simple')
cat.remove('sku_config')
cont.remove('id_customer')
cont.append('bust')
cont.append('waist')
cont.append('hip')
cat.remove('target_size_actual')
cat.remove('bust')
cat.remove('waist')
cat.remove('hip')
cont, cat

(['size_le', 'bust', 'waist', 'hip'],
 ['gender',
  'venture_code',
  'sub_category_type',
  'size_name',
  'size_system_name',
  'brand_name',
  'product_gender',
  'department_code'])

In [40]:
'''
device =torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

# feed the data into the Learner and train
to_nn = TabularPandas(df2, procs, cat, cont,
                      splits=splits, y_names='target_size')


dls = TabularPandas(df2, y_names="target_size",
    cat_names = cat,
    cont_names = cont, splits = RandomSplitter(valid_pct=0.2)(range_of(df2)))


to_nn = dls.dataloaders()


dls = to_nn.dataloaders(1024, device = device)
'''

'\ndevice =torch.device(\'cuda\' if torch.cuda.is_available() else \'cpu\')\nprint(device)\n\n# feed the data into the Learner and train\nto_nn = TabularPandas(df2, procs, cat, cont,\n                      splits=splits, y_names=\'target_size\')\n\n\ndls = TabularPandas(df2, y_names="target_size",\n    cat_names = cat,\n    cont_names = cont, splits = RandomSplitter(valid_pct=0.2)(range_of(df2)))\n\n\nto_nn = dls.dataloaders()\n\n\ndls = to_nn.dataloaders(1024, device = device)\n'

In [41]:
#learn = tabular_learner(to_nn, metrics=accuracy)
#learn.lr_find()'''

In [42]:
#learn.fit_one_cycle(1, 2e-2)

In [43]:
#learn.export('fastai_v1_tabular_v1.pkl')

In [44]:
#to_nn.train.xs.head()

In [45]:
'''def embed_features(learner, xs):
    xs = xs.copy()
    for i, feature in enumerate(learner.dls.cat_names):
        emb = learner.model.embeds[i]
        new_feat = pd.DataFrame(emb(tensor(xs[feature], dtype=torch.int64)), index=xs.index, columns=[f'{feature}_{j}' for j in range(emb.embedding_dim)])
        xs.drop(columns=feature, inplace=True)
        xs = xs.join(new_feat)
    return xs
emb_xs = embed_features(learn, to_nn.train.xs)
emb_valid_xs = embed_features(learn, to_nn.valid.xs)'''

"def embed_features(learner, xs):\n    xs = xs.copy()\n    for i, feature in enumerate(learner.dls.cat_names):\n        emb = learner.model.embeds[i]\n        new_feat = pd.DataFrame(emb(tensor(xs[feature], dtype=torch.int64)), index=xs.index, columns=[f'{feature}_{j}' for j in range(emb.embedding_dim)])\n        xs.drop(columns=feature, inplace=True)\n        xs = xs.join(new_feat)\n    return xs\nemb_xs = embed_features(learn, to_nn.train.xs)\nemb_valid_xs = embed_features(learn, to_nn.valid.xs)"

In [46]:
#emb_xs.head()

In [47]:
random.seed(1)

# Shuffle dataframe
df2 = df2.sample(frac = 1)

train_data = df2[:int(0.8*len(df2))]
val_data = df2[int(0.8*len(df2)):int(0.9*len(df2))]
test_data = df2[int(0.9*len(df2)):]

len(train_data), len(val_data), len(test_data)

(66504, 8313, 8313)

In [48]:
for i,r in train_data[0:10].iterrows():
  print(r['sku_le'])

3303
3965
1000
610
4479
1391
1949
2769
2085
1391


## Index user and **items**

In [49]:

item_data = {}
item_index = {}
user_index = {}
user_data = {}
u_index = 0
i_index = 0
for index, r in train_data.iterrows():
    if str(r['sku_le']) + '|' + str(r['size_le']) not in item_data:
        item_data[str(r['sku_le']) + '|' + str(r['size_le'])] = [r]
        item_index[str(r['sku_le']) + '|' + str(r['size_le'])] = i_index
        i_index += 1
    else:
        item_data[str(r['sku_le']) + '|' + str(r['size_le'])].append(r)

    if r['id_customer'] not in user_data:
        user_data[r['id_customer']] = [r]
        user_index[r['id_customer']] = u_index
        u_index += 1
    else:
        user_data[r['id_customer']].append(r)

In [50]:
len(user_data), len(user_index), len(item_data), len(item_index)

(43803, 43803, 12178, 12178)

In [51]:
#item_data

In [52]:
true_size_item = np.zeros(len(item_data))
true_size_cust = np.zeros(len(user_data))
#w = 1; b_1 = -1; b_2 = 1; lamda = 2

for item in item_data:
    true_size_item[item_index[item]] = int(item.split('|')[1])

In [53]:
true_size_item[:10]

array([43., 43., 21., 52., 43., 21., 57., 19., 19., 17.])

In [54]:
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import label_binarize

In [55]:
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
import xgboost as xgb

In [56]:
def data_proc(data):

  features = []
  labels = []
  for i,r in data.iterrows():
    fe = []
    fe.append(true_size_cust[user_index[r['id_customer']]])
    fe.append(true_size_item[item_index[str(r['sku_le']) + '|' + str(r['size_le'])]])
    features.append(fe)

    if 'small' in r['target_size_actual']:
        labels.append(0)
    elif 'fit' in r['target_size_actual']:
        labels.append(1)
    elif 'large' in r['target_size_actual']:
        labels.append(2)

  return features, labels

In [57]:
train_features, train_labels = data_proc(train_data)

In [58]:
c = 1
clf_1LV = XGBClassifier(random_state=1, n_estimators=500, nfold=5, metrics=["mlogloss", "auc"])
clf_1LV.fit(train_features, train_labels)

[06:11:43] WARNING: ../src/learner.cc:767: 
Parameters: { "metrics", "nfold" } are not used.



XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              metrics=['mlogloss', 'auc'], min_child_weight=None, missing=nan,
              monotone_constraints=None, n_estimators=500, n_jobs=None, nfold=5,
              num_parallel_tree=None, ...)

In [59]:
'''clf_1LV.save_model("model.json")
clf_1LV = xgb.Booster()

clf_1LV.load_model("model.json")'''


import pickle

pickle.dump(clf_1LV, open("xgb_reg.pkl", "wb"))
clf_1LV = pickle.load(open("xgb_reg.pkl", "rb"))

In [60]:
product.head(1)

,sku,sku_simple,size_name,size_system_name,measurements,bust,waist,hip,product_gender,brand_name,department_code
0,IN509AA81SXUSG,IN509AA81SXUSG-445250,XS,International,,38,31,41,Female,inner circle,RBO


In [61]:
## Import test_data
prod_df1 = df1[['sku_config']].drop_duplicates()

product = product[(product['sku'].isin(prod_df1['sku_config']))]
test_cust = df1[(df1['id_customer'].isin(test_data['id_customer'])) & (~df1['id_customer'].isin(train_data['id_customer']))][['id_customer', 'venture_code', 'gender', 'customer_age']].drop_duplicates()
test_cust.head()

,id_customer,venture_code,gender,customer_age
2361,5824464,ph,female,23.0000
2688,3639184,ph,,NaN
3651,8847269,ph,female,NaN
5939,743023,sg,female,NaN
6401,1590129,my,female,29.0000


In [62]:
test_cust = test_cust.sample(5)
product = product.sample(10)

In [63]:
product['sub_category_type'] = 'Dresses'

In [64]:
test_data = test_cust.join(product,how='cross')
test_data.rename(columns={'sku':'sku_config'},inplace=True)


In [65]:
test_data['size_le'] = le.fit_transform(test_data['size_name'])
test_data['sku_le'] = le.fit_transform(test_data['sku_config'])
test_data = test_data[train_data.drop(['target_size_actual', 'target_size'],axis=1).columns]
test_data.head()

,id_customer,gender,venture_code,sku_config,sku_simple,sub_category_type,size_name,size_system_name,brand_name,product_gender,department_code,bust,waist,hip,size_le,sku_le
0,1208556,female,my,MI511AA0SP6TMY,MI511AA0SP6TMY-2276499,Dresses,6,UK,missguided,Female,RBO,39,33,41,1,9
1,1208556,female,my,FF025AA1197CB3GS,FF025AA1197CB3GS-041TFU,Dresses,XL,International,anna alba,Female,LBC,51,38,44,5,8
2,1208556,female,my,78468AA73FEA4AGS,78468AA73FEA4AGS-089LCC,Dresses,XS,International,zalora,Female,RPL,38,32.5,44.5,6,3
3,1208556,female,my,7919CAA6DB3D90GS,7919CAA6DB3D90GS-099TVS,Dresses,42,EU,trendyol,Female,RBO,50,42,52,0,4
4,1208556,female,my,E9AD9AABBF50B7GS,E9AD9AABBF50B7GS-077DKG,Dresses,XS,International,something borrowed,Female,RPL,46.5,31.5,54.5,6,7


In [66]:
## tEST DATA
test_features = []; test_labels = []; test_labels_auc = []
label = [0, 0, 0]
for i,r in test_data.iterrows():

  fe = []
  try:
      fe.append(true_size_cust[user_index[r['id_customer']]])
  except KeyError:
      fe.append(np.mean(true_size_cust))
  try:
      fe.append(true_size_item[item_index[str(r['sku_le']) + '|' + str(r['size_le'])]])
  except KeyError:
      fe.append(np.mean(true_size_item))

  test_features.append(fe)
'''  label = [0, 0, 0]
  if 'small' in r['target_size_actual']:
      test_labels.append(0)
      label[0] = 1
  elif 'fit' in r['target_size_actual']:
      test_labels.append(1)
      label[1] = 1
  elif 'large' in r['target_size_actual']:
      test_labels.append(2)
      label[2] = 1'''
  #test_labels_auc.append(label)

#test_labels_auc = np.array(test_labels_auc)

"  label = [0, 0, 0]\n  if 'small' in r['target_size_actual']:\n      test_labels.append(0)\n      label[0] = 1\n  elif 'fit' in r['target_size_actual']:\n      test_labels.append(1)\n      label[1] = 1\n  elif 'large' in r['target_size_actual']:\n      test_labels.append(2)\n      label[2] = 1"

In [67]:
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import label_binarize

pred = clf_1LV.predict_proba(test_features)
'''AUC = []
for i in range(3):
    AUC.append(roc_auc_score(test_labels_auc[:,i], pred[:,i], average='weighted'))
print('Average AUC', np.mean(AUC), AUC)'''

"AUC = []\nfor i in range(3):\n    AUC.append(roc_auc_score(test_labels_auc[:,i], pred[:,i], average='weighted'))\nprint('Average AUC', np.mean(AUC), AUC)"

In [68]:
pred = pd.DataFrame(pred,columns=['small', 'fit', 'large'])

In [69]:
pred.head(15)

,small,fit,large
0,0.0904,0.8174,0.0922
1,0.0904,0.8174,0.0922
2,0.0904,0.8174,0.0922
3,0.0904,0.8174,0.0922
4,0.0904,0.8174,0.0922
5,0.0904,0.8174,0.0922
6,0.0904,0.8174,0.0922
7,0.0904,0.8174,0.0922
8,0.0904,0.8174,0.0922
9,0.0904,0.8174,0.0922


In [70]:
len(pred[pred['fit']>0.8])/len(pred)

1.0

In [71]:
test_data = test_data.reset_index()
test_data = pd.concat([test_data,pred],axis=1)

In [72]:
test_data.head(20)

,index,id_customer,gender,venture_code,sku_config,sku_simple,sub_category_type,size_name,size_system_name,brand_name,product_gender,department_code,bust,waist,hip,size_le,sku_le,small,fit,large
0,0,1208556,female,my,MI511AA0SP6TMY,MI511AA0SP6TMY-2276499,Dresses,6,UK,missguided,Female,RBO,39,33,41,1,9,0.0904,0.8174,0.0922
1,1,1208556,female,my,FF025AA1197CB3GS,FF025AA1197CB3GS-041TFU,Dresses,XL,International,anna alba,Female,LBC,51,38,44,5,8,0.0904,0.8174,0.0922
2,2,1208556,female,my,78468AA73FEA4AGS,78468AA73FEA4AGS-089LCC,Dresses,XS,International,zalora,Female,RPL,38,32.5,44.5,6,3,0.0904,0.8174,0.0922
3,3,1208556,female,my,7919CAA6DB3D90GS,7919CAA6DB3D90GS-099TVS,Dresses,42,EU,trendyol,Female,RBO,50,42,52,0,4,0.0904,0.8174,0.0922
4,4,1208556,female,my,E9AD9AABBF50B7GS,E9AD9AABBF50B7GS-077DKG,Dresses,XS,International,something borrowed,Female,RPL,46.5,31.5,54.5,6,7,0.0904,0.8174,0.0922
5,5,1208556,female,my,09CF2AA490981DGS,09CF2AA490981DGS-092MZB,Dresses,M,International,zalora,Female,RPL,44,40.5,50.5,3,0,0.0904,0.8174,0.0922
6,6,1208556,female,my,DC1C6AA166216CGS,DC1C6AA166216CGS-092LSD,Dresses,S,International,zalora,Female,RPL,43,33,48.5,4,6,0.0904,0.8174,0.0922
7,7,1208556,female,my,9BD9DAA11542A0GS,9BD9DAA11542A0GS-000ZSX,Dresses,L,International,zalora,Female,RPL,46,39.5,53.5,2,5,0.0904,0.8174,0.0922
8,8,1208556,female,my,5F033AA7D952E0GS,5F033AA7D952E0GS-006OYJ,Dresses,S,International,get laud,Female,LBC,42,36,64,4,2,0.0904,0.8174,0.0922
9,9,1208556,female,my,1D1E5AA9DE5DBAGS,1D1E5AA9DE5DBAGS-036OPR,Dresses,L,International,something borrowed,Female,RPL,40.5,36.5,55.5,2,1,0.0904,0.8174,0.0922


In [73]:
df1[df1['id_customer']==3975102][['sku_simple', 'brand_name', 'size_name']].drop_duplicates()

,sku_simple,brand_name,size_name
2475298,75C9EAA62E28C0GS-070FGN,Trendyol,M


In [74]:
product[product['sku_simple']=='6F0B6AA6253B39GS-087AVW']

,sku,sku_simple,size_name,size_system_name,measurements,bust,waist,hip,product_gender,brand_name,department_code,sub_category_type


In [75]:
test_data['pred_final'] = np.where(test_data['fit']>0.6,'fit','not fit')

In [76]:
test_data = test_data[test_data['pred_final']=='fit'].sort_values(by=['bust','waist', 'hip'])[['id_customer', 'sku_config', 'sku_simple', 'size_name']].drop_duplicates()

In [77]:
test1 = test_data.groupby(['id_customer', 'sku_config'])['size_name'].nunique().reset_index()
test1.rename(columns={'size_name':'size_count'},inplace=True)

In [78]:
test1['size_count'] = np.ceil(test1['size_count']/2)

test_data['C'] = test_data.sort_values(by=['id_customer', 'sku_config','size_name']).groupby(['id_customer', 'sku_config']).cumcount()+1


test_data = pd.merge(test_data,test1,on=['id_customer', 'sku_config'],how='left')
test_data = test_data[test_data['C']==test_data['size_count']]

test_data.head()

,id_customer,sku_config,sku_simple,size_name,C,size_count
0,1208556,78468AA73FEA4AGS,78468AA73FEA4AGS-089LCC,XS,1,1.0000
1,3343422,78468AA73FEA4AGS,78468AA73FEA4AGS-089LCC,XS,1,1.0000
2,9590940,78468AA73FEA4AGS,78468AA73FEA4AGS-089LCC,XS,1,1.0000
3,3733913,78468AA73FEA4AGS,78468AA73FEA4AGS-089LCC,XS,1,1.0000
4,324746,78468AA73FEA4AGS,78468AA73FEA4AGS-089LCC,XS,1,1.0000


In [83]:
## Shares list of customers and their fit sizes for every sku
test_data.to_csv('Test_customer_fit_sizes.csv', index=None)